# Data augmentation

Data augmentation is the process of altering original data items in a structured way to produce similar data items that differ from the original data item in a precisely defined way. Data augmentation techniques can be applied to textual data, image data, and audio data. In this exercise we will focus on augmentation techniques for text and images.

## Augmentation of text

Data augmentation for text can be performed on three levels:
- character modifications
- word modifications
- sentence modifications

### Character-level augmentation

There are two primary reasons for doing character-level augmentation:
- to simulate typing errors
- to simulate OCR errors

Below we start with a simple example of augmentation that tries to address these two cases.

In [ ]:
from nlpaug.augmenter.char import KeyboardAug

_input = 'Poznan is a medium-sized city on the banks of the Warta river'
_output = KeyboardAug().augment(_input)

print(f"{_input}\n{_output}")

The `nlpaug` library allows to produce multiple augmentations for a single input, as well as processing a list of texts.

In [ ]:
_input = 'To be or not to be, this is the question'
_output = KeyboardAug().augment(_input, n=5)

print(f"{_input}")

for result in _output:
    print(f"{result}")

In [ ]:
_input = [
    'This is the first sentence',
    'This is the second sentence',
    'This is the third sentence',
]
_output = KeyboardAug().augment(_input)

for original, transformed in zip(_input, _output):
    print(f"{original}\n{transformed}\n")

If the training data for the ML model have been extracted via OCR from images, there is probably a very specific bias in the data. For instance, the lower case letter `l` can be easily mistaken for the digit `1` or for the upper case `I`. Similarly, `9` and `g` are often confused as well as `0` and `O`. `OCRAug` class simulates these types of mistakes.

In [ ]:
from nlpaug.augmenter.char import OcrAug

_input = 'This bottle has the volume of 10l and it weights 230g'
_output = OcrAug().augment(_input)

print(f"{_input}\n{_output}")

Finally, `nlpaug` allows to introduce random insertions, swaps, and deletions of characters from the source text.

In [ ]:
from nlpaug.augmenter.char import RandomCharAug
from nlpaug.flow import Action

_input = 'Imagine all the people living life of peace'

_output_ins = RandomCharAug(action=Action.INSERT).augment(_input)
_output_sub = RandomCharAug(action='substitute').augment(_input)
_output_swp = RandomCharAug(action='swap').augment(_input)
_output_del = RandomCharAug(action='delete').augment(_input)

print(f"{_input}")
print(f"{_output_ins}\n{_output_sub}\n{_output_swp}\n{_output_del}")

### Word-level augmentation

Word-level augmentation can be used in many scenarios depending on the characteristics of the training dataset. Examples of augmentations include:

- substituting a word for an alternative spelling
- finding a synonym of a word based on word embeddings
- finding a synonym of a word based on TF-IDF
- finding a synonym of a word based on contextual word embeddings
- finding a synonym of a word based on a dictionary
- randomly splitting words
- substituting a word for its antonym

Below we will see some of the above scenarios in action.

In [ ]:
from nlpaug.augmenter.word import SpellingAug

_input = 'In a hole in the ground there lived a hobbit'

_output = SpellingAug().augment(_input, n=3)

print(f"{_input}")

for result in _output:
    print(f"{result}")

The next example requires you to download the pre-trained word embeddings. We are using `fasttext` Englishl embeddings trained on the Wikipedia corpus. You can download the model from [https://fasttext.cc/docs/en/english-vectors.html](https://fasttext.cc/docs/en/english-vectors.html)

In [ ]:
from nlpaug.augmenter.word import WordEmbsAug
from gensim.models import KeyedVectors

augmenter = WordEmbsAug(model_type='fasttext', 
                        model_path='wiki-news-300d-1M.vec',
                        action='substitute',
                        top_k=50
                       )

In [ ]:
_input = 'To be or not to be this is the question'
_output = augmenter.augment(_input)

print(f"{_input}\n{_output}")

`FastText` or `word2vec` embeddings are static, a given word always receives the same vector, independent of the textual context. If you want to use contextual embeddings, you have to use the BERT family of language models (`bert-base-uncased`, `distilbert-base-uncased`, `roberta-base`, etc.)

In [ ]:
from nlpaug.augmenter.word import ContextualWordEmbsAug

augmenter = ContextualWordEmbsAug(
    model_path='roberta-base', 
    action='substitute'
)

In [ ]:
_input = 'In a hole in the ground there lived a hobbit'
_output = augmenter.augment(_input, n=10)

print(f"{_input}\n{_output}")

In [ ]:
from nlpaug.augmenter.word import SynonymAug

_input = 'In the hole in the ground there lived a little hobbit'
_output = SynonymAug(aug_src='wordnet').augment(_input)

print(f"{_input}\n{_output}")

An interesting option is to use antonyms for generating alternatives for the training data. Here is a simple example.

In [ ]:
from nlpaug.augmenter.word import AntonymAug

_input = 'I really loved the movie it was great'
_output = AntonymAug().augment(_input)

print(f"{_input}\n{_output}")

Finally, `RandomWordAug` allows us to randomly swap, delete and crop words from the input.

In [ ]:
from nlpaug.augmenter.word import RandomWordAug

_input = 'Once upon a time there was a little bird'

_output_swp = RandomWordAug(action='swap').augment(_input)
_output_del = RandomWordAug(action='delete').augment(_input)
_output_crp = RandomWordAug(action='crop').augment(_input)

print(f"{_input}")
print(f"{_output_swp}\n{_output_del}\n{_output_crp}")

There is also a word-level augmenter which performs random splitting of words.

In [ ]:
from nlpaug.augmenter.word import SplitAug

_input = 'In a hole in the ground there lived a hobbit'
_output = SplitAug().augment(_input)

print(f"{_input}\n{_output}")

Finally, if you want to augment only specific words (for which you know precisely the alternatives), you can use the `ReservedAug`.

In [ ]:
from nlpaug.augmenter.word import ReservedAug

reserved_words = [
    ['FW', 'Fwd', 'F/D', 'Forward'],
    ['Q1', 'Q2', 'Q3', 'Q4']
]

_input = 'Fwd: Sales report for Q1'
_output = ReservedAug(reserved_tokens=reserved_words).augment(_input)

print(f"{_input}\n{_output}")

### Sentence-level augmentation

Sentence-level augmentation uses state-of-the-art contextual word embeddings created by the BERT family of language models. In order to use these models you have to install the following dependencies:

```bash
bash$ pip install torch>=1.6.0 transformers>=4.0.0
```

In [ ]:
from nlpaug.augmenter.sentence import ContextualWordEmbsForSentenceAug

augmenter = ContextualWordEmbsForSentenceAug(
    model_path='gpt2' , 
    min_length=10,
    temperature=0.5, 
    top_k=50,
    top_p=0.7
)

_input = 'I really enjoyed the wine'
_output = augmenter.augment(_input)

print(f"{_input}\n{_output}")

### Sequences of augmentations

`nlpaug` allows you to define two modes of sequential augmentations:
- provide a list of augmentations to be applied sequentially
- provide a list of augmentations which will randomly be applied

In [ ]:
from nlpaug.flow import Sequential, Sometimes
from nlpaug.util import Action
from nlpaug.augmenter.word import RandomWordAug
from nlpaug.augmenter.char import RandomCharAug

_flow = Sequential(
    [
    RandomCharAug(action=Action.INSERT),
    RandomCharAug(action=Action.SWAP),
    RandomWordAug(action=Action.DELETE),
    ]
)

_input = 'In a hole in the ground there lived a hobbit'
_output = _flow.augment(_input)

print(f"{_input}\n{_output}")

### Exercise

Use the [IMDB Dataset](https://www.kaggle.com/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews) to create a sample of 100 movie reviews. Then, create a sequence of transformations that are relevant for training the sentiment model. Assume that the reviews have been manually typed on a keyboard. Include character-level, word-level and sentence-level augmentations. In particular, try to augment movie reviews with relevant synonyms. 

## Augmentation for images

We will experiment with `Augmentor` library to perform random augmentations of images. `Augmentor` allows you to define a _pipeline_ of augmentations. Each augmentation has a parameter `probability` which defines, how probable it is that the augmentation will be applied. 

For this part of the exercise we will use the [Face Mask Detection dataset](https://www.kaggle.com/sshikamaru/face-mask-detection) from Kaggle Datasets.

After downloading and unpacking the dataset, please create an additional directory with a single image for demonstration purposes.

```bash
bash$ mkdir images/example
bash$ cp images/train/smartmi-3pcs-filter-mask-pm25-haze-dustproof-mask-with-vent_jpg.rf.c5b0c5b7666032c5b4634740eafde234.jpg images/example/image.jpg
```

In [ ]:
import Augmentor

path_to_files = 'images/example/'

p = Augmentor.Pipeline(path_to_files)
p.status()

We will start with a very simple horizontal mirror flip of images

In [ ]:
p.flip_left_right(probability=0.5)

p.sample(5)

In [ ]:
from IPython.display import Image, display, HTML

from glob import glob

def make_html(image):
     return '<img src="{}" style="display:inline;margin:1px;width:100px"/>'.format(image)
    
original_image = 'images/example/image.jpg'
transformed_images = glob('images/example/output/*.jpg')

display(Image(filename=original_image, width=100, height=100))

html_images = ''.join([make_html(x) for x in transformed_images])
display(HTML(html_images))    

Next, we will illustrate some more transformations available in the `Augmentor` library

In [ ]:
!rm images/example/output/*

In [ ]:
# rotation

path_to_files = 'images/example'

p = Augmentor.Pipeline(path_to_files)

# p.rotate(probability=1.0, max_left_rotation=25, max_right_rotation=25)
# p.rotate180(probability=0.5)
p.rotate90(probability=0.5)
# p.rotate_random_90(probability=0.5)

p.sample(10)

original_image = 'images/example/image.jpg'
transformed_images = glob('images/example/output/*.jpg')

display(Image(filename=original_image, width=100, height=100))

html_images = ''.join([make_html(x) for x in transformed_images])
display(HTML(html_images))

In [ ]:
!rm images/example/output/*

In [ ]:
# zooming

path_to_files = 'images/example'

p = Augmentor.Pipeline(path_to_files)

# p.zoom(probability=0.9, min_factor=1.1, max_factor=3.0)
p.zoom_random(probability=0.9, percentage_area=0.5)

p.sample(10)

original_image = 'images/example/image.jpg'
transformed_images = glob('images/example/output/*.jpg')

display(Image(filename=original_image, width=100, height=100))

html_images = ''.join([make_html(x) for x in transformed_images])
display(HTML(html_images))

In [ ]:
!rm images/example/output/*

In [ ]:
# perspective skewing

path_to_files = 'images/example'

p = Augmentor.Pipeline(path_to_files)

p.skew_tilt(probability=0.9)
# p.skew_left_right(probability=0.5)
# p.skew_top_bottom(probability=0.5)
# p.skew_corner(probability=0.5)

p.sample(10)

original_image = 'images/example/image.jpg'
transformed_images = glob('images/example/output/*.jpg')

display(Image(filename=original_image, width=100, height=100))

html_images = ''.join([make_html(x) for x in transformed_images])
display(HTML(html_images))

In [ ]:
!rm images/example/output/*

In [ ]:
# elastic distortions

path_to_files = 'images/example'

p = Augmentor.Pipeline(path_to_files)

# p.random_distortion(probability=0.9, grid_width=50, grid_height=50, magnitude=10)
# p.random_color(probability=0.5, min_factor=0.1, max_factor=0.5)
# p.random_contrast(probability=0.5, min_factor=0.1, max_factor=0.5)
# p.random_erasing(probability=0.5, rectangle_area=0.25)
p.random_brightness(probability=0.5, min_factor=0.1, max_factor=0.5)

p.sample(10)

original_image = 'images/example/image.jpg'
transformed_images = glob('images/example/output/*.jpg')

display(Image(filename=original_image, width=100, height=100))

html_images = ''.join([make_html(x) for x in transformed_images])
display(HTML(html_images))

In [ ]:
!rm images/example/output/*

In [ ]:
# shearing

path_to_files = 'images/example'

p = Augmentor.Pipeline(path_to_files)

p.shear(probability=0.5, max_shear_left=25, max_shear_right=25)

p.sample(10)

original_image = 'images/example/image.jpg'
transformed_images = glob('images/example/output/*.jpg')

display(Image(filename=original_image, width=100, height=100))

html_images = ''.join([make_html(x) for x in transformed_images])
display(HTML(html_images))

In [ ]:
!rm images/example/output/*

In [ ]:
# cropping

path_to_files = 'images/example'

p = Augmentor.Pipeline(path_to_files)

# p.crop_centre(probability=0.9, percentage_area=0.75)
# p.crop_by_size(probability=0.5, width=100, height=100)
p.crop_random(probability=0.9, percentage_area=0.5)

p.sample(10)

original_image = 'images/example/image.jpg'
transformed_images = glob('images/example/output/*.jpg')

display(Image(filename=original_image, width=100, height=100))

html_images = ''.join([make_html(x) for x in transformed_images])
display(HTML(html_images))

If you wish to process each image in the pipeline exactly once, use `process()`

In [ ]:
!rm images/example/output/*

In [ ]:
# resizing

def make_html_original_size(image):
     return '<img src="{}" style="display:inline;margin:1px"/>'.format(image)
    
path_to_files = 'images/example'

p = Augmentor.Pipeline(path_to_files)

p.resize(probability=1.0, width=100, height=100)

p.process()

original_image = 'images/example/image.jpg'
transformed_images = glob('images/example/output/*.jpg')

display(Image(filename=original_image))

html_images = ''.join([make_html_original_size(x) for x in transformed_images])
display(HTML(html_images))

### Exercise

Use the `images/train` folder to create a set of transformations of images. When designing the augmentation pipeline, consider the main aim of the augmentation: to improve the model for face mask detection.

## Testing NLP models with Checklist

Checklist is an interesting library which allows you to perform behavioral testing of NLP models. Behavioral tests do not verify the internal structure of the model, they simply verify the behavior (compare output to expected output) of the model. Checklist allows to do two basic tasks:

- create artificial test instances (examples)
- apply augmentations and distortions to test instances

We begin by using a *template* to create some test data. The template may simply insert words from a dictionary into a placeholder, or it can use a special token `{mask}` to insert arbitrary tokens.

In [1]:
import checklist
from checklist.editor import Editor

pos_adjectives = ["good", "great", "amazing", "super", "fantastic"]

e = Editor()

e.template("This is a recipe for a {a:pos} dinner.", pos=pos_adjectives, nsamples=3).data

['This is a recipe for a a good dinner.',
 'This is a recipe for a an amazing dinner.',
 'This is a recipe for a a fantastic dinner.']

In [2]:
e.template("{mask} is not {a:pos} recipe for dinner.", pos=pos_adjectives, nsamples=3).data

/home/mikolaj/venv/python39/lib/python3.9/site-packages/checklist/text_generation.py:171: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:245.)
  to_pred = torch.tensor(to_pred, device=self.device).to(torch.int64)


['Apple is not a great recipe for dinner.',
 'Music is not an amazing recipe for dinner.',
 'Today is not a fantastic recipe for dinner.']

We will now create a larger test suite of generic phrases with affective adjectives.

In [4]:
import random

pos = [
  "good", "realistic", "healthy", "attractive", "appealing", "acceptable", 
  "best", "feasible", "easy", "ideal", "affordable", "economical", "recommended", 
  "exciting", "inexpensive", "obvious", "great", "appropriate", "effective", "excellent",
  ]

neg = [
  "bad", "unhealthy", "expensive", "boring", "terrible", "worst", "unfeasible", 
  "unappropriate", "awful", "time-consuming",
  ]
  
samples = e.template("For holidays {mask} is a {a:pos} option.", pos=pos, labels=0, save=True, nsamples=100)
samples += e.template("For holidays {mask} is a {a:neg} option.", neg=neg, labels=1, save=True, nsamples=100)

list(zip(samples.data, samples.labels))

[('For holidays gardening is a a realistic option.', 0),
 ('For holidays breastfeeding is a a recommended option.', 0),
 ('For holidays vaping is a a feasible option.', 0),
 ('For holidays skiing is a an exciting option.', 0),
 ('For holidays sailing is a a best option.', 0),
 ('For holidays volunteering is a an effective option.', 0),
 ('For holidays there is a an acceptable option.', 0),
 ('For holidays rent is a an effective option.', 0),
 ('For holidays football is a an economical option.', 0),
 ('For holidays fishing is a an affordable option.', 0),
 ('For holidays printing is a an inexpensive option.', 0),
 ('For holidays Pizza is a an effective option.', 0),
 ('For holidays sailing is a an appealing option.', 0),
 ('For holidays cider is a an ideal option.', 0),
 ('For holidays VPN is a an appropriate option.', 0),
 ('For holidays bacon is a a good option.', 0),
 ('For holidays chicken is a an exciting option.', 0),
 ('For holidays Android is a an ideal option.', 0),
 ('For holi

For sentiment classification we will use a simple class `TextBlob` from the `textblob` module.

In [5]:
from textblob import TextBlob

TextBlob("This pizza was so good one could kill for it").sentiment

Sentiment(polarity=0.7, subjectivity=0.6000000000000001)

We will also create a simple utility function to return positive/negative sentiment probability

In [6]:
import numpy as np

def predict_proba(inputs):
    p1 = np.array([(TextBlob(x).sentiment[0] + 1) / 2.0 for x in inputs]).reshape(-1, 1)
    p0 = 1 - p1
    
    return np.hstack((p0, p1))


predict_proba(["This pizza was so good one could kill for it"])

array([[0.15, 0.85]])

In order to use our probability prediction function we need to pass it through the wrapper.

In [7]:
from checklist.pred_wrapper import PredictorWrapper

wrapped_pp = PredictorWrapper.wrap_softmax(predict_proba)
wrapped_pp(["This pizza was so good one could kill for it"])

(array([1]), array([[0.15, 0.85]]))

There are three basic types of tests available in `checklist`:

- **minimum functionality test** (MFT): uses simple examples to make sure the model can perform a specific task well
- **invariance test** (INV): checks if the model prediction stays the same when trivial parts of inputs are slightly changed
- **directional expectation test** (DET): checks if the model changes the prediction in the desired direction when parts of the inputs are slightly changed

In [8]:
from checklist.test_types import MFT, INV, DIR

test = MFT(
    samples.data,
    labels=samples.labels,
    name="Test negation",
    capability="Negation"
)

test.run(wrapped_pp)

Predicting 200 examples


In [9]:
test.summary()

Test cases:      200
Fails (rate):    169 (84.5%)

Example fails:
1.0 For holidays turkey is a a best option.
----
0.5 For holidays medication is a an unappropriate option.
----
0.7 For holidays flying is a an exciting option.
----


In [10]:
test.visual_summary()

TestSummarizer(stats={'npassed': 31, 'nfailed': 169, 'nfiltered': 0}, summarizer={'name': 'Test negation', 'de…

To perform invariance and directional expectation tests we have to convert the sample data into `spacy` documents. We will use a small set of fictitious reviews of restaurants.

In [11]:
data = [
    "The falafel was delicious.",
    "Everything was awful and there were flies in the soup!",
    "It was the best food me and Kathy in a long time.",
    "Jimmy hates lettuce...",
    "This burger joint is not very good?",
    "Restaurants in Germany are always great!"
]

In [14]:
import spacy

nlp = spacy.load("en_core_web_sm")
spacy_data = list(nlp.pipe(data))

spacy_data

[The falafel was delicious.,
 Everything was awful and there were flies in the soup!,
 It was the best food me and Kathy in a long time.,
 Jimmy hates lettuce...,
 This burger joint is not very good?,
 Restaurants in Germany are always great!]

First, we will verify if the model correctly predicts the sentiment when we perturb the punctuation.

In [15]:
from checklist.perturb import Perturb

_data = Perturb.perturb(spacy_data, Perturb.punctuation)

for row in _data.data:
    print(row)

['The falafel was delicious.', 'The falafel was delicious']
['Everything was awful and there were flies in the soup!', 'Everything was awful and there were flies in the soup', 'Everything was awful and there were flies in the soup.']
['It was the best food me and Kathy in a long time.', 'It was the best food me and Kathy in a long time']
['Jimmy hates lettuce...', 'Jimmy hates lettuce', 'Jimmy hates lettuce.']
['This burger joint is not very good?', 'This burger joint is not very good', 'This burger joint is not very good.']
['Restaurants in Germany are always great!', 'Restaurants in Germany are always great', 'Restaurants in Germany are always great.']


In [16]:
test  = INV(**_data)
test.run(wrapped_pp)

test.summary()
test.visual_summary()

Predicting 16 examples
Test cases:      6
Fails (rate):    0 (0.0%)


TestSummarizer(stats={'npassed': 6, 'nfailed': 0, 'nfiltered': 0}, summarizer={'name': None, 'description': No…

Next, we will add some typos.

In [17]:
_data = Perturb.perturb(data, Perturb.add_typos)

for row in _data.data:
    print(row)

['The falafel was delicious.', 'The falafel was delicoius.']
['Everything was awful and there were flies in the soup!', 'Everything was afwul and there were flies in the soup!']
['It was the best food me and Kathy in a long time.', 'It was the best food me and aKthy in a long time.']
['Jimmy hates lettuce...', 'Jimmyh ates lettuce...']
['This burger joint is not very good?', 'This burgerj oint is not very good?']
['Restaurants in Germany are always great!', 'Restaurants in Germanya re always great!']


In [18]:
test  = INV(**_data)
test.run(wrapped_pp)

test.summary()
test.visual_summary()

Predicting 12 examples
Test cases:      6
Fails (rate):    1 (16.7%)

Example fails:
1.0 The falafel was delicious.
0.5 The falafel was delicoius.

----


TestSummarizer(stats={'npassed': 5, 'nfailed': 1, 'nfiltered': 0}, summarizer={'name': None, 'description': No…

We should verify if the predictions change when we change the proper names.

In [24]:
for d in spacy_data:
    print(d, [(e.text, e.label_) for e in d.ents])

The falafel was delicious. []
Everything was awful and there were flies in the soup! []
It was the best food me and Kathy in a long time. [('Kathy', 'PERSON')]
Jimmy hates lettuce... [('Jimmy', 'PERSON')]
This burger joint is not very good? []
Restaurants in Germany are always great! [('Germany', 'GPE')]


In [19]:
_data = Perturb.perturb(spacy_data, Perturb.change_names)

for row in _data.data:
    print(row)

['It was the best food me and Kathy in a long time.', 'It was the best food me and Elizabeth in a long time.', 'It was the best food me and Destiny in a long time.', 'It was the best food me and Heather in a long time.', 'It was the best food me and Chelsea in a long time.', 'It was the best food me and Katie in a long time.', 'It was the best food me and Victoria in a long time.', 'It was the best food me and Christine in a long time.', 'It was the best food me and Brittany in a long time.', 'It was the best food me and Tracy in a long time.', 'It was the best food me and Shannon in a long time.']
['Jimmy hates lettuce...', 'Michael hates lettuce...', 'Thomas hates lettuce...', 'Jeremiah hates lettuce...', 'Isaac hates lettuce...', 'Samuel hates lettuce...', 'Justin hates lettuce...', 'Steven hates lettuce...', 'Timothy hates lettuce...', 'Stephen hates lettuce...', 'Antonio hates lettuce...']


In [25]:
test  = INV(**_data)
test.run(wrapped_pp)

test.summary()
test.visual_summary()

Predicting 22 examples
Test cases:      2
Fails (rate):    0 (0.0%)


TestSummarizer(stats={'npassed': 2, 'nfailed': 0, 'nfiltered': 0}, summarizer={'name': None, 'description': No…

We also do not expect any prediction changes in response to changing the names of cities or countries

In [26]:
_data = Perturb.perturb(spacy_data, Perturb.change_location)

for row in _data.data:
    print(row)

['Restaurants in Germany are always great!', 'Restaurants in Yemen are always great!', 'Restaurants in Bangladesh are always great!', 'Restaurants in Italy are always great!', 'Restaurants in Turkey are always great!', 'Restaurants in Uzbekistan are always great!', 'Restaurants in Argentina are always great!', 'Restaurants in Ethiopia are always great!', 'Restaurants in Vietnam are always great!', 'Restaurants in Peru are always great!', 'Restaurants in Uganda are always great!']


In [27]:
test  = INV(**_data)
test.run(wrapped_pp)

test.summary()
test.visual_summary()

Predicting 11 examples
Test cases:      1
Fails (rate):    0 (0.0%)


TestSummarizer(stats={'npassed': 1, 'nfailed': 0, 'nfiltered': 0}, summarizer={'name': None, 'description': No…

The last type of test assumes that the change in the input example will trigger the change in the model's prediction and the direction of the change is known. Since we are doing binary sentiment classification only, the direction of change is simply the prediction of the opposite sentiment. We begin by defining a helper function to flag the change of prediction.

In [28]:
from checklist.expect import Expect

def changed_pred(orig_pred, pred, orig_conf, conf, labels=None, meta=None):
    return pred != orig_pred


expect_fn = Expect.pairwise(changed_pred)

As a simple example, we will add negation to our test cases.

In [30]:
_data = Perturb.perturb(spacy_data, Perturb.add_negation)

In [31]:
_data

MunchWithAdd({'data': [['The falafel was delicious.', 'The falafel was not delicious.'], ['Everything was awful and there were flies in the soup!', 'Everything was not awful and there were flies in the soup!'], ['It was the best food me and Kathy in a long time.', 'It was not the best food me and Kathy in a long time.'], ['Jimmy hates lettuce...', "Jimmy doesn't hate lettuce..."], ['Restaurants in Germany are always great!', 'Restaurants in Germany are not always great!']]})

In [32]:
for row in _data.data:
    print(row)

['The falafel was delicious.', 'The falafel was not delicious.']
['Everything was awful and there were flies in the soup!', 'Everything was not awful and there were flies in the soup!']
['It was the best food me and Kathy in a long time.', 'It was not the best food me and Kathy in a long time.']
['Jimmy hates lettuce...', "Jimmy doesn't hate lettuce..."]
['Restaurants in Germany are always great!', 'Restaurants in Germany are not always great!']


In [34]:
test = DIR(**_data, expect=expect_fn)
test.run(wrapped_pp)

test.summary()
test.visual_summary()

Predicting 10 examples
Test cases:      5
Fails (rate):    3 (60.0%)

Example fails:
0.5 Jimmy hates lettuce...
0.1 Jimmy doesn't hate lettuce...

----
1.0 Restaurants in Germany are always great!
1.0 Restaurants in Germany are not always great!

----
0.7 It was the best food me and Kathy in a long time.
0.7 It was not the best food me and Kathy in a long time.

----


TestSummarizer(stats={'npassed': 2, 'nfailed': 3, 'nfiltered': 0}, summarizer={'name': None, 'description': No…

### Exercise

Prepare a list of five short movie reviews. Write one invariance test and one directionality expectation test. Check if the `TextBlob` sentiment classifier can pass your tests.